In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis


import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00   1.03 MB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00   1.59 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   3.29 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  31.54 MB/s
vincent-0.4.4- 100% |###################

In [118]:
# I am extracting the neighbourhoods of Mumbai from the above link used.you can see the extracted webpage.
import unicodedata
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of neighbourhoods in Mumbai - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_neighbourhoods_in_Mumbai","wgTitle":"List of neighbourhoods in Mumbai","wgCurRevisionId":859956824,"wgRevisionId":859956824,"wgArticleId":37060396,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 errors: dates","CS1 maint: Unfit url","Neighbourhoods in Mumbai","Lists of neighbourhoods in Indian cities","Mumbai-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransf

In [119]:
#All the hyperlinks which has our neighboorhood name is extracted and first converted to list and then pandas series which you can see.
import pandas as pd
main = soup.ul
second = []
second = main.find_all('a')
my_list = list(second)
s = pd.Series(my_list)
s

0     <a href="#Western_Suburbs"><span class="tocnum...
1     <a href="#Andheri"><span class="tocnumber">1.1...
2     <a href="#Bandra"><span class="tocnumber">1.2<...
3     <a href="#Borivali"><span class="tocnumber">1....
4     <a href="#Dahisar"><span class="tocnumber">1.4...
5     <a href="#Goregaon"><span class="tocnumber">1....
6     <a href="#Jogeshwari"><span class="tocnumber">...
7     <a href="#Juhu"><span class="tocnumber">1.7</s...
8     <a href="#Kandivali_west"><span class="tocnumb...
9     <a href="#Khar"><span class="tocnumber">1.9</s...
10    <a href="#Malad"><span class="tocnumber">1.10<...
11    <a href="#Santacruz"><span class="tocnumber">1...
12    <a href="#Vile_Parle"><span class="tocnumber">...
13    <a href="#Eastern_Suburbs"><span class="tocnum...
14    <a href="#Bhandup_HOP"><span class="tocnumber"...
15    <a href="#Ghatkopar"><span class="tocnumber">2...
16    <a href="#Kanjurmarg"><span class="tocnumber">...
17    <a href="#Kurla"><span class="tocnumber">2

In [120]:
# The previous list elements are split based on the split info.
# After splitting it is converted into a dataframe. 
k = []
for i in range(len(my_list)):
    k.append((str(my_list[i])).split('<span class="toctext">')[1].split('</span></a>')[0])

df = pd.DataFrame(k)
df

,0
0,Western Suburbs
1,Andheri
2,Bandra
3,Borivali
4,Dahisar
5,Goregaon
6,Jogeshwari
7,Juhu
8,Kandivali west
9,Khar


In [121]:
# the dropped elements are not neighbourhoods but the sectors of neighboorhoods. thats why they are dropped. you can check the webpage. 
# Also the last 3 elements from dataframe which are not neighbourhoods are also dropped.
df.drop([0,13,22,27,39,40,41],inplace=True)

#the area names in dataframe are set as index for further analysis
df.index = df[0]

#Extra columns named longitude and latitude are added with o value so that we can update it in next steps.
df['Latitude'] = 0
df['Longitude'] = 0


In [122]:
#Columns names are formatted properly.
df.columns = ['Area','Latitude','Longitude']

In [123]:
#you can see the resultant dataframe.
df

,Area,Latitude,Longitude
0,,,
Andheri,Andheri,0,0
Bandra,Bandra,0,0
Borivali,Borivali,0,0
Dahisar,Dahisar,0,0
Goregaon,Goregaon,0,0
Jogeshwari,Jogeshwari,0,0
Juhu,Juhu,0,0
Kandivali west,Kandivali west,0,0
Khar,Khar,0,0


In [124]:
# foursquare API does not provide the latitudes of Bhandup HOP neighbourhood so we had to remove it as it was considered as null in dataframe
df.loc['Bhandup HOP','Area'] = 'Bhandup'

#Now using the geolocator API we add the latitudes and longitudes of every neighbourhood area in the dataframe. 
for i in df['Area']:
    address = (i +' ' +'Mumbai')
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    latitude = df.loc[i,'Latitude'] = location.latitude
    longitude = df.loc[i,'Longitude'] = location.longitude




/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [126]:
#you can see the resultant dataframe here with areas and its latitudes and longitudes
df

,Area,Latitude,Longitude
0,,,
Andheri,Andheri,19.120371,72.848043
Bandra,Bandra,19.054979,72.840220
Borivali,Borivali,19.228738,72.856877
Dahisar,Dahisar,19.257178,72.857536
Goregaon,Goregaon,19.164973,72.849543
Jogeshwari,Jogeshwari,19.135734,72.848923
Juhu,Juhu,19.107021,72.827528
Kandivali west,Kandivali west,19.208380,72.842227
Khar,Khar,19.072457,72.833707


In [127]:
# create map of mumbai city neighbourhoods from the previous dataframe using latitude and longitude values 
# This looks as if neighbourhoods are superimposed on Mumbai city.
# You can zoom in and out to see the map.

map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Area in zip(df['Latitude'], df['Longitude'], df['Area']):
    label = '{}, {}'.format(Area,"Mumbai")
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)  
    
map_mumbai

In [12]:
#credentials set up for foursquare use
CLIENT_ID = '1QVU4DYVKRQNMKQEXUJPMV14R2ESCMSCOB3S4ETRHHVQUFOL' # your Foursquare ID
CLIENT_SECRET = 'A3AIOGYTXJVA3JKIKUAEAOUNDQZTXUS2QCNEQCWYKH51H4D1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1QVU4DYVKRQNMKQEXUJPMV14R2ESCMSCOB3S4ETRHHVQUFOL
CLIENT_SECRET:A3AIOGYTXJVA3JKIKUAEAOUNDQZTXUS2QCNEQCWYKH51H4D1


In [128]:
# I am setting parameters for the query i am going to do using foursquare api.
# I am a reseracher so i want to know at which neighbourhood in mumbai city i can live so maximum research places are near my neighbourhood. 
# thats why i set the query for reserch. you guys can look for anything or look for more specific places. 
# There is no accurate center point of mumbai but kurla place is approximately at center. So i used kurla's latitide and longitude values to begin search from.
# From the center, the API will look for reserach places within 50,000 meters which covers almost whole mumbai city.
# limit is set to 100 but in reality foursquare only provides 50 results.


lat = 19.065059
lon = 72.879242
search_query = 'Research'
radius = 50000 
LIMIT = 100

In [28]:
#query string which is used in request to foursquare API
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=1QVU4DYVKRQNMKQEXUJPMV14R2ESCMSCOB3S4ETRHHVQUFOL&client_secret=A3AIOGYTXJVA3JKIKUAEAOUNDQZTXUS2QCNEQCWYKH51H4D1&ll=18.9672753,72.8221514&v=20180605&query=Research&radius=50000&limit=100'

In [129]:
#sending the request to foursquare API and storing in result variable.
results = requests.get(url).json()

#The result we get in resonse, is in json so we select the venue information from it and convert into the dataframe.
venues = results['response']['venues']
dataframe = json_normalize(venues)

#you can see the venue information converted in dataframe format.
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'shortName': 'Hospital', 'name': 'Hospital',...",False,55928d98498e3756c360c5aa,Raja Rammohan Roy Road,IN,Mumbai,India,"Prarthana Samaj, Girgaon",963,"[Raja Rammohan Roy Road (Prarthana Samaj, Girg...","[{'label': 'display', 'lng': 72.8202338236881,...",18.958816,72.820234,400004,Mahārāshtra,Sir H. N. Reliance Foundation Hospital and Res...,v-1542786312
1,"[{'shortName': 'Spiritual', 'name': 'Spiritual...",False,4fd864e2e4b0de27c8d39966,S.V. P. Road,IN,Mumbai,India,Dongri,1674,"[S.V. P. Road (Dongri), Mumbai 400009, Mahārās...","[{'label': 'display', 'lng': 72.83400177955627...",18.957242,72.834002,400009,Mahārāshtra,Islamic Research Foundation (IRF),v-1542786312
2,"[{'shortName': 'Hospital', 'name': 'Hospital',...",False,4ccffbcc2fd9b60c21567eeb,NaN,IN,Mumbai,India,Peddar Road,1376,"[Mumbai 400026, Mahārāshtra, India]","[{'label': 'display', 'lng': 72.809925, 'lat':...",18.971656,72.809925,400026,Mahārāshtra,Jaslok Hospital & Research Centre,v-1542786312
3,"[{'shortName': 'Academic Building', 'name': 'C...",False,4b0587e5f964a52018a622e3,"Lakhamshi Napoo Road,",IN,Mumbai,India,Matunga CR,7084,"[Lakhamshi Napoo Road, (Matunga CR), Mumbai 40...","[{'label': 'display', 'lng': 72.85031414835852...",19.025082,72.850314,400019,Mahārāshtra,WE School - Prin. LN Welingkar Institute Of Ma...,v-1542786312
4,"[{'shortName': 'Hospital', 'name': 'Hospital',...",False,4b0587e5f964a5202ca622e3,"A-791, Bandra Reclamation",IN,Mumbai,India,Bandra,9342,"[A-791, Bandra Reclamation (Bandra), Mumbai 40...","[{'label': 'display', 'lng': 72.8294454684838,...",19.050915,72.829445,400 050,Mahārāshtra,Lilavati Hospital & Research Center,v-1542786312
5,"[{'shortName': 'Lab', 'name': 'College Lab', '...",False,4e9e45e76da161b08d83d9e2,Sarvoday Buddh Vihar Marg,IN,Mumbai,India,Vidyavihar,14302,"[Sarvoday Buddh Vihar Marg (Vidyavihar), Mumba...","[{'label': 'display', 'lng': 72.89979618407213...",19.072727,72.899796,NaN,Mahārāshtra,Research. Innovation. Incubation. Design Lab (...,v-1542786312
6,"[{'shortName': 'Hospital', 'name': 'Hospital',...",False,4cb28da91168a093681a3c23,Somaiya Ayurvihar Complex,IN,Mumbai,India,Sion,10442,"[Somaiya Ayurvihar Complex (Sion), Mumbai 4000...","[{'label': 'display', 'lng': 72.87462001931617...",19.046896,72.874620,400023,Mahārāshtra,K J Somaiya Medical College And Research Centre,v-1542786312
7,"[{'shortName': 'Medical Lab', 'name': 'Medical...",False,4fe9616be4b0f2679d8ca28c,NaN,IN,NaN,India,NaN,1572,[India],"[{'label': 'display', 'lng': 72.8359146118164,...",18.972763,72.835915,NaN,NaN,research laboratory,v-1542786312
8,"[{'shortName': 'Medical Lab', 'name': 'Medical...",False,4fe961d2e4b09fac02184398,NaN,IN,NaN,India,NaN,1601,[India],"[{'label': 'display', 'lng': 72.836468899794, ...",18.972144,72.836469,NaN,NaN,research laboratory,v-1542786312
9,"[{'shortName': 'Capitol Building', 'name': 'Ca...",False,51066c7ce4b022bf0ad7b1a3,NaN,IN,NaN,India,NaN,1016,[India],"[{'label': 'display', 'lng': 72.82247161865234...",18.958149,72.822472,NaN,NaN,Family Welfare Training & Research Centre,v-1542786312


In [30]:
# we keep only columns that include venue name, and anything that is associated with location so that our results become more accurate 

filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#the name of venues in as index of dataframe for easy further analysis.
dataframe_filtered.index = dataframe_filtered['name']

#You can the resultant dataframe
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
name,,,,,,,,,,,,,,,
Sir H. N. Reliance Foundation Hospital and Research Centre,Sir H. N. Reliance Foundation Hospital and Res...,Hospital,Raja Rammohan Roy Road,IN,Mumbai,India,"Prarthana Samaj, Girgaon",963,"[Raja Rammohan Roy Road (Prarthana Samaj, Girg...","[{'label': 'display', 'lng': 72.8202338236881,...",18.958816,72.820234,400004,Mahārāshtra,55928d98498e3756c360c5aa
Islamic Research Foundation (IRF),Islamic Research Foundation (IRF),Spiritual Center,S.V. P. Road,IN,Mumbai,India,Dongri,1674,"[S.V. P. Road (Dongri), Mumbai 400009, Mahārās...","[{'label': 'display', 'lng': 72.83400177955627...",18.957242,72.834002,400009,Mahārāshtra,4fd864e2e4b0de27c8d39966
Jaslok Hospital & Research Centre,Jaslok Hospital & Research Centre,Hospital,NaN,IN,Mumbai,India,Peddar Road,1376,"[Mumbai 400026, Mahārāshtra, India]","[{'label': 'display', 'lng': 72.809925, 'lat':...",18.971656,72.809925,400026,Mahārāshtra,4ccffbcc2fd9b60c21567eeb
WE School - Prin. LN Welingkar Institute Of Management Development & Research,WE School - Prin. LN Welingkar Institute Of Ma...,College Academic Building,"Lakhamshi Napoo Road,",IN,Mumbai,India,Matunga CR,7084,"[Lakhamshi Napoo Road, (Matunga CR), Mumbai 40...","[{'label': 'display', 'lng': 72.85031414835852...",19.025082,72.850314,400019,Mahārāshtra,4b0587e5f964a52018a622e3
Lilavati Hospital & Research Center,Lilavati Hospital & Research Center,Hospital,"A-791, Bandra Reclamation",IN,Mumbai,India,Bandra,9342,"[A-791, Bandra Reclamation (Bandra), Mumbai 40...","[{'label': 'display', 'lng': 72.8294454684838,...",19.050915,72.829445,400 050,Mahārāshtra,4b0587e5f964a5202ca622e3
Research. Innovation. Incubation. Design Lab (riddl),Research. Innovation. Incubation. Design Lab (...,College Lab,Sarvoday Buddh Vihar Marg,IN,Mumbai,India,Vidyavihar,14302,"[Sarvoday Buddh Vihar Marg (Vidyavihar), Mumba...","[{'label': 'display', 'lng': 72.89979618407213...",19.072727,72.899796,NaN,Mahārāshtra,4e9e45e76da161b08d83d9e2
K J Somaiya Medical College And Research Centre,K J Somaiya Medical College And Research Centre,Hospital,Somaiya Ayurvihar Complex,IN,Mumbai,India,Sion,10442,"[Somaiya Ayurvihar Complex (Sion), Mumbai 4000...","[{'label': 'display', 'lng': 72.87462001931617...",19.046896,72.874620,400023,Mahārāshtra,4cb28da91168a093681a3c23
research laboratory,research laboratory,Medical Lab,NaN,IN,NaN,India,NaN,1572,[India],"[{'label': 'display', 'lng': 72.8359146118164,...",18.972763,72.835915,NaN,NaN,4fe9616be4b0f2679d8ca28c
research laboratory,research laboratory,Medical Lab,NaN,IN,NaN,India,NaN,1601,[India],"[{'label': 'display', 'lng': 72.836468899794, ...",18.972144,72.836469,NaN,NaN,4fe961d2e4b09fac02184398


In [130]:
# from above dataframe we only need name, latitude, longitude and id of venue so we select it. 
dataframe_filtered = dataframe_filtered[["name", "lat", "lng",'id']]

In [131]:
# the extracted features dataframe
dataframe_filtered

,name,lat,lng,id
name,,,,
Sir H. N. Reliance Foundation Hospital and Research Centre,Sir H. N. Reliance Foundation Hospital and Res...,18.958816,72.820234,55928d98498e3756c360c5aa
Islamic Research Foundation (IRF),Islamic Research Foundation (IRF),18.957242,72.834002,4fd864e2e4b0de27c8d39966
Jaslok Hospital & Research Centre,Jaslok Hospital & Research Centre,18.971656,72.809925,4ccffbcc2fd9b60c21567eeb
WE School - Prin. LN Welingkar Institute Of Management Development & Research,WE School - Prin. LN Welingkar Institute Of Ma...,19.025082,72.850314,4b0587e5f964a52018a622e3
Lilavati Hospital & Research Center,Lilavati Hospital & Research Center,19.050915,72.829445,4b0587e5f964a5202ca622e3
Research. Innovation. Incubation. Design Lab (riddl),Research. Innovation. Incubation. Design Lab (...,19.072727,72.899796,4e9e45e76da161b08d83d9e2
K J Somaiya Medical College And Research Centre,K J Somaiya Medical College And Research Centre,19.046896,72.874620,4cb28da91168a093681a3c23
research laboratory,research laboratory,18.972763,72.835915,4fe9616be4b0f2679d8ca28c
research laboratory,research laboratory,18.972144,72.836469,4fe961d2e4b09fac02184398


In [132]:
# create map of venues using latitude and longitude values and it is superimposed on our earlier map of mumbai neighbourhoods  .

map_mumbai_venues = folium.Map(location=[latitude, longitude], zoom_start=10)

# The green ones are research places of mumbai and blue ones are the neighbourhoods of mumbai.

# add markers to map
for lat, lng, name in zip(dataframe_filtered['lat'], dataframe_filtered['lng'], dataframe_filtered['name']):
    label = '{}, {}'.format(name,"Mumbai_venues")
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)  
    
map_mumbai

In [136]:
# here we create an empty dataframe with reserach places as index and neighbourhoods as features or columns 
Dist_matrix = pd.DataFrame(columns = df.index, index = dataframe_filtered.index)
Dist_matrix

,Andheri,Bandra,Borivali,Dahisar,Goregaon,Jogeshwari,Juhu,Kandivali west,Khar,Malad,...,Byculla,Colaba,Dadar,Fort,Girgaon,Kalbadevi,Kamathipura,Matunga,Parel,Tardeo
name,,,,,,,,,,,,,,,,,,,,,
Sir H. N. Reliance Foundation Hospital and Research Centre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Islamic Research Foundation (IRF),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jaslok Hospital & Research Centre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WE School - Prin. LN Welingkar Institute Of Management Development & Research,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Lilavati Hospital & Research Center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Research. Innovation. Incubation. Design Lab (riddl),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
K J Somaiya Medical College And Research Centre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
research laboratory,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
research laboratory,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
# Now we are calculating the distance of each mumbai neighbourhood from each of the research places we got. 
# this is the formula for calculating the distance between two points where each point has latitude and longitude 

from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
R = 6373.0
g=0.00
for p in df['Area']:
    
    for q in range (dataframe_filtered.shape[0]):
        lat1 = radians(df.loc[p,'Latitude'])
        lon1 = radians(df.loc[p,'Longitude'])
        lat2 = radians(dataframe_filtered.iloc[q,1])
        lon2 = radians(dataframe_filtered.iloc[q,2])

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        
#we update the distance in distance matrix as we iterate over each neighbourhood with each research place.
# we store the distances in distance matrix as well as print it.

        Dist_matrix.loc[Dist_matrix.index[q],p] = distance = R * c
        
        print("Distance from "+ str(p) + " to " + dataframe_filtered.iloc[q,0] + " is " + str(distance)+ "  km")
         


Distance from Andheri to Sir H. N. Reliance Foundation Hospital and Research Centre is 18.20616284189792  km
Distance from Andheri to Islamic Research Foundation (IRF) is 18.20481565088139  km
Distance from Andheri to Jaslok Hospital & Research Centre is 17.020163881323004  km
Distance from Andheri to WE School - Prin. LN Welingkar Institute Of Management Development & Research is 10.601687784833892  km
Distance from Andheri to Lilavati Hospital & Research Center is 7.969086312612053  km
Distance from Andheri to Research. Innovation. Incubation. Design Lab (riddl) is 7.5943707273152645  km
Distance from Andheri to K J Somaiya Medical College And Research Centre is 8.636960273381076  km
Distance from Andheri to research laboratory is 16.46788640868302  km
Distance from Andheri to research laboratory is 16.532113704589147  km
Distance from Andheri to Family Welfare Training & Research Centre is 18.243185669808334  km
Distance from Andheri to Aditya Birla Centre for Performing Arts & Rese

In [138]:
#This is the distance matrix which has distance of each neighbourhood from each of the research places.
Dist_matrix

,Andheri,Bandra,Borivali,Dahisar,Goregaon,Jogeshwari,Juhu,Kandivali west,Khar,Malad,...,Byculla,Colaba,Dadar,Fort,Girgaon,Kalbadevi,Kamathipura,Matunga,Parel,Tardeo
name,,,,,,,,,,,,,,,,,,,,,
Sir H. N. Reliance Foundation Hospital and Research Centre,18.2062,10.9008,30.2695,33.4176,23.1369,19.9085,16.5028,27.8551,12.7195,25.1758,...,2.38081,4.90076,7.50725,3.21451,0.557025,1.33684,0.974343,8.25045,5.87612,0.96236
Islamic Research Foundation (IRF),18.2048,10.891,30.2941,33.4535,23.1636,19.9155,16.6739,27.9474,12.8154,25.2983,...,2.15937,4.764,7.42774,2.66733,1.724,1.09345,1.20059,7.98064,5.77789,1.67316
Jaslok Hospital & Research Centre,17.0202,9.80032,29.018,32.1504,21.9022,18.7053,15.17,26.5488,11.4876,23.85,...,2.46806,6.5142,6.57999,4.9925,2.10357,3.13011,1.87412,7.50942,5.0682,1.3753
WE School - Prin. LN Welingkar Institute Of Management Development & Research,10.6017,3.4907,22.6632,25.8272,15.5602,12.3087,9.42366,20.4059,5.55131,17.8171,...,5.69647,12.4994,1.15338,10.347,8.57749,8.75617,7.10569,0.24737,2.23802,7.07926
Lilavati Hospital & Research Center,7.96909,1.21969,19.9882,23.1317,12.8612,9.65397,6.24395,17.5662,2.43766,14.8933,...,8.27113,15.1121,3.191,13.0969,10.813,11.3085,9.52574,3.42463,4.72707,9.33483
Research. Innovation. Incubation. Design Lab (riddl),7.59437,6.56693,17.9295,20.9914,11.5399,8.81508,8.50033,16.2562,6.94764,14.1519,...,12.8028,19.1762,8.3592,16.9636,15.7362,15.6753,14.2209,7.24913,9.65117,14.2914
K J Somaiya Medical College And Research Centre,8.63696,3.72679,20.3121,23.4585,13.3955,10.244,8.32063,18.2816,5.15607,15.859,...,8.96917,15.528,4.50332,13.3243,11.9002,11.9185,10.3926,3.35553,5.74467,10.4345
research laboratory,16.5321,9.22215,28.6214,31.7815,21.4923,18.2431,15.0318,26.2834,11.1616,23.6401,...,0.630475,6.44145,5.75661,4.32926,2.78286,2.69923,1.29992,6.30692,4.1104,1.60046
research laboratory,16.5321,9.22215,28.6214,31.7815,21.4923,18.2431,15.0318,26.2834,11.1616,23.6401,...,0.630475,6.44145,5.75661,4.32926,2.78286,2.69923,1.29992,6.30692,4.1104,1.60046


In [93]:
# now we want to find out which research places are near which neighbourhoods.
# i used the assumption that near means less than 2.5 kilometers.
# you can choose any assumption you may like. 

m = Dist_matrix[Dist_matrix <=2.5]

#Only the dataframe values which are less than or equal to 2.5 kilometers will be shown in dataframe. Other will be null. 
m  

,Andheri,Bandra,Borivali,Dahisar,Goregaon,Jogeshwari,Juhu,Kandivali west,Khar,Malad,...,Byculla,Colaba,Dadar,Fort,Girgaon,Kalbadevi,Kamathipura,Matunga,Parel,Tardeo
name,,,,,,,,,,,,,,,,,,,,,
Sir H. N. Reliance Foundation Hospital and Research Centre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.380812,NaN,NaN,NaN,0.557025,1.336844,0.974343,NaN,NaN,0.962360
Islamic Research Foundation (IRF),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.159372,NaN,NaN,NaN,1.723996,1.093450,1.200593,NaN,NaN,1.673156
Jaslok Hospital & Research Centre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.468064,NaN,NaN,NaN,2.103567,NaN,1.874121,NaN,NaN,1.375297
WE School - Prin. LN Welingkar Institute Of Management Development & Research,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.153378,NaN,NaN,NaN,NaN,0.247370,2.238018,NaN
Lilavati Hospital & Research Center,NaN,1.219685,NaN,NaN,NaN,NaN,NaN,NaN,2.437664,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Research. Innovation. Incubation. Design Lab (riddl),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
K J Somaiya Medical College And Research Centre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
research laboratory,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.630475,NaN,NaN,NaN,NaN,NaN,1.299922,NaN,NaN,1.600460
research laboratory,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.630475,NaN,NaN,NaN,NaN,NaN,1.299922,NaN,NaN,1.600460


In [103]:
# we substract the number of null places of that series/column from the number of elements in each series/column.
# the result is we get the number of research places that are near to each neighbourhood.  

50 - m.isnull().sum() 

0
Andheri            0
Bandra             4
Borivali           1
Dahisar            1
Goregaon           0
Jogeshwari         0
Juhu               0
Kandivali west     1
Khar               2
Malad              0
Santacruz          1
Vile Parle         0
Ghatkopar          3
Kanjurmarg         0
Kurla              6
Mulund             0
Powai              0
Vidyavihar         4
Vikhroli           0
Chembur            3
Govandi            1
Mankhurd           1
Trombay            1
Antop Hill         6
Byculla            7
Colaba             6
Dadar             12
Fort               8
Girgaon           11
Kalbadevi         11
Kamathipura       11
Matunga            8
Parel             10
Tardeo            10
dtype: int64

<B><h1>In above results, you can see neighbourhood named 'Dadar ' has the highest number of research places so the most suitable place to live if you are a reserach person will be Dadar. 
Yet the neighbourhoods Girgaon, Kalbadevi, Kamathipura are also very near the research places so you can also live over there too.

I wanted to use the tips or rating of the venues to decide which ones are the best venues and then select only top 50 venues and then do this project to find the best neighbourhood to live so you could have access to most venues. 
But the above code shows that it was not possible as people dont use foursqaure application in Mumbai . As a result there are no tips or ratings of the venues in mumbai. 
Thats why I needed to go with the first 50 venues I get from foursquare API and then decide based on the distance which neighbourhood is best to live so you could have access to maximum venues. 

In [139]:
lat = 19.120371       
lon = 72.848043
search_query = 'Research'
radius = 50000 
LIMIT = 100

In [140]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=1QVU4DYVKRQNMKQEXUJPMV14R2ESCMSCOB3S4ETRHHVQUFOL&client_secret=A3AIOGYTXJVA3JKIKUAEAOUNDQZTXUS2QCNEQCWYKH51H4D1&ll=18.9672753,72.8221514&v=20180605&query=Research&radius=50000&limit=100'

In [141]:
results_new = requests.get(url).json()
venues_new = results_new['response']['venues']

dataframe_new = json_normalize(venues_new)

In [142]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_new = ['name', 'categories'] + [col for col in dataframe_new.columns if col.startswith('location.')] + ['id']
dataframe_filtered_new = dataframe_new.loc[:, filtered_columns]


# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_new['categories'] = dataframe_filtered_new.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_new.columns = [column.split('.')[-1] for column in dataframe_filtered_new.columns]

dataframe_filtered_new

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Sir H. N. Reliance Foundation Hospital and Res...,Hospital,Raja Rammohan Roy Road,IN,Mumbai,India,"Prarthana Samaj, Girgaon",963,"[Raja Rammohan Roy Road (Prarthana Samaj, Girg...","[{'label': 'display', 'lng': 72.8202338236881,...",18.958816,72.820234,400004,Mahārāshtra,55928d98498e3756c360c5aa
1,Islamic Research Foundation (IRF),Spiritual Center,S.V. P. Road,IN,Mumbai,India,Dongri,1674,"[S.V. P. Road (Dongri), Mumbai 400009, Mahārās...","[{'label': 'display', 'lng': 72.83400177955627...",18.957242,72.834002,400009,Mahārāshtra,4fd864e2e4b0de27c8d39966
2,Jaslok Hospital & Research Centre,Hospital,NaN,IN,Mumbai,India,Peddar Road,1376,"[Mumbai 400026, Mahārāshtra, India]","[{'label': 'display', 'lng': 72.809925, 'lat':...",18.971656,72.809925,400026,Mahārāshtra,4ccffbcc2fd9b60c21567eeb
3,WE School - Prin. LN Welingkar Institute Of Ma...,College Academic Building,"Lakhamshi Napoo Road,",IN,Mumbai,India,Matunga CR,7084,"[Lakhamshi Napoo Road, (Matunga CR), Mumbai 40...","[{'label': 'display', 'lng': 72.85031414835852...",19.025082,72.850314,400019,Mahārāshtra,4b0587e5f964a52018a622e3
4,Lilavati Hospital & Research Center,Hospital,"A-791, Bandra Reclamation",IN,Mumbai,India,Bandra,9342,"[A-791, Bandra Reclamation (Bandra), Mumbai 40...","[{'label': 'display', 'lng': 72.8294454684838,...",19.050915,72.829445,400 050,Mahārāshtra,4b0587e5f964a5202ca622e3
5,Research. Innovation. Incubation. Design Lab (...,College Lab,Sarvoday Buddh Vihar Marg,IN,Mumbai,India,Vidyavihar,14302,"[Sarvoday Buddh Vihar Marg (Vidyavihar), Mumba...","[{'label': 'display', 'lng': 72.89979618407213...",19.072727,72.899796,NaN,Mahārāshtra,4e9e45e76da161b08d83d9e2
6,K J Somaiya Medical College And Research Centre,Hospital,Somaiya Ayurvihar Complex,IN,Mumbai,India,Sion,10442,"[Somaiya Ayurvihar Complex (Sion), Mumbai 4000...","[{'label': 'display', 'lng': 72.87462001931617...",19.046896,72.874620,400023,Mahārāshtra,4cb28da91168a093681a3c23
7,research laboratory,Medical Lab,NaN,IN,NaN,India,NaN,1601,[India],"[{'label': 'display', 'lng': 72.836468899794, ...",18.972144,72.836469,NaN,NaN,4fe961d2e4b09fac02184398
8,research laboratory,Medical Lab,NaN,IN,NaN,India,NaN,1572,[India],"[{'label': 'display', 'lng': 72.8359146118164,...",18.972763,72.835915,NaN,NaN,4fe9616be4b0f2679d8ca28c
9,Family Welfare Training & Research Centre,Capitol Building,NaN,IN,NaN,India,NaN,1016,[India],"[{'label': 'display', 'lng': 72.82247161865234...",18.958149,72.822472,NaN,NaN,51066c7ce4b022bf0ad7b1a3


In [143]:
for i in dataframe_filtered['id']:
    venue_id = i
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result_venues = requests.get(url).json()
    try:
        print(result['response']['venue']['tips']['count'])
    except:
        print('This venue does not have tips yet.')

This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not have tips yet.
This venue does not 